# Issue 994

https://github.com/astronomy-commons/lsdb/issues/994

I'm trying to poke at this issue, and find the point where it breaks down.

But I don't think it's an LSDB problem - this looks like a subtle thing within nested, and now this notebook has turned into my attempt to pinpoint it.


In [40]:
import lsdb

lsdb.__version__

'0.7.1'

In [57]:
ztf_22_scattered = lsdb.open_catalog('/epyc/data3/hats/catalogs/ztf_dr22', search_filter = lsdb.ConeSearch(ra=285, dec=-15, radius_arcsec=1*3600))
ztf_22 = ztf_22_scattered.map_partitions(lambda nf: nf.reset_index().set_index("objectid").nest_lists(columns=['hmjd', 'mag', 'magerr', 'catflags', 'clrcoeff'], 
                                                                    name="lc").reset_index().set_index("_healpix_29"))

## Side by side frames

I'm going to pull the meta frame, and also a materialized dataframe from the catalog, and perform identical operations on both frames. You'll see the point where they diverge.

In [58]:
nf_m = ztf_22_scattered._ddf.meta

In [59]:
nf = ztf_22_scattered.partitions[0].compute()

In [60]:
new_f = nf.reset_index().set_index("objectid").nest_lists(columns=['hmjd', 'mag', 'magerr', 'catflags', 'clrcoeff'], name="lc")#.reset_index()#.set_index("_healpix_29")
new_f.head(1)

_healpix_29  filterid       objra     objdec  \
objectid                                                                 
1377101100038760  2044346982812312015         1  285.162384 -15.987551   

                  nepochs                                                 lc  
objectid                                                                      
1377101100038760        2  [{hmjd: 59820.26301, mag: 19.223154, magerr: 0...

In [61]:
new_m_f = nf_m.reset_index().set_index("objectid").nest_lists(columns=['hmjd', 'mag', 'magerr', 'catflags', 'clrcoeff'],name="lc")#.reset_index()#.set_index("_healpix_29")
new_m_f

,_healpix_29,filterid,objra,objdec,nepochs,lc


In [62]:
new_f.columns == new_m_f.columns

array([ True,  True,  True,  True,  True,  True])

## Here it is

Calling `reset_index` on the empty catalog results in different columns. But not when `reset_index` was getting called on an empty frame above??

In [63]:
new_f = nf.reset_index().set_index("objectid").nest_lists(columns=['hmjd', 'mag', 'magerr', 'catflags', 'clrcoeff'],name="lc").reset_index()#.set_index("_healpix_29")
new_f.head(1)

objectid          _healpix_29  filterid       objra     objdec  \
0  1377101100038760  2044346982812312015         1  285.162384 -15.987551   

   nepochs                                                 lc  
0        2  [{hmjd: 59820.26301, mag: 19.223154, magerr: 0...

In [64]:
new_m_f = nf_m.reset_index().set_index("objectid").nest_lists(columns=['hmjd', 'mag', 'magerr', 'catflags', 'clrcoeff'],name="lc").reset_index()#.set_index("_healpix_29")
new_m_f

,index,_healpix_29,filterid,objra,objdec,nepochs,lc


In [56]:
new_f.columns == new_m_f.columns

array([False,  True,  True,  True,  True,  True,  True])